In [1]:
import scanpy as sc
import pandas as pd 
import numpy as np
import anndata
import re
import h5py
import scipy.sparse as scs
import concurrent.futures
import seaborn as sns
from scipy.sparse import csc_matrix
import random
random.seed(123)
from joblib import Parallel, delayed
from concurrent.futures import ThreadPoolExecutor, as_completed


In [2]:
meta_data=pd.read_csv('/home/jupyter/formal_BRIanalysis/DEG_Mehul/hise_meta_data_2024-01-23_fixed.csv')
meta_data=meta_data[meta_data['Covid_exlcusion']=='no']
TimePoints=['Flu Year 1 Day 0', 'Flu Year 1 Day 7', 'Flu Year 1 Day 90', 'Flu Year 2 Day 0', 'Flu Year 2 Day 7', 'Flu Year 2 Day 90']
meta_data_subset=meta_data[meta_data['sample.visitName'].isin(TimePoints)]

In [3]:
from tqdm import tqdm
h5_list = []
for i in tqdm(range(len(meta_data_subset["pbmc_sample_id"].tolist()))):
    try:
        result = sc.read_h5ad("/home/jupyter/h5_cleaned_by_sample/"+meta_data_subset["pbmc_sample_id"].tolist()[i]+'.h5ad')
        h5_list.append(result)
        print()
    except Exception as e:
        print(f'Error reading {file_name}: {e}')

  0% 1/512 [00:00<02:11,  3.89it/s]

  0% 2/512 [00:00<02:06,  4.05it/s]

  1% 3/512 [00:00<02:05,  4.06it/s]

  1% 4/512 [00:02<05:31,  1.53it/s]

  1% 5/512 [00:04<09:48,  1.16s/it]

...

 99% 508/512 [15:50<00:06,  1.57s/it]

 99% 509/512 [15:52<00:05,  1.71s/it]

100% 510/512 [15:53<00:03,  1.67s/it]

100% 511/512 [15:55<00:01,  1.67s/it]

100% 512/512 [15:56<00:00,  1.87s/it]

In [4]:
%%time
combined= anndata.concat(h5_list)

/opt/conda/lib/python3.10/site-packages/anndata/_core/merge.py:217: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if pd.api.types.is_categorical_dtype(dtype):


CPU times: user 19min 4s, sys: 2min 35s, total: 21min 40s
Wall time: 21min 40s


In [5]:
%%time
combined.write_h5ad('/home/jupyter/formal_BRIanalysis/DEG_Mehul/for_3times_2yr.h5ad')

CPU times: user 3min 33s, sys: 50.9 s, total: 4min 24s
Wall time: 16min 24s


In [9]:
%%time
combined=sc.read_h5ad('/home/jupyter/formal_BRIanalysis/DEG_Mehul/for_immunevar_Day_0_7.h5ad')

CPU times: user 11.9 s, sys: 23.9 s, total: 35.9 s
Wall time: 35.9 s


In [10]:
combined.obs.index=combined.obs['barcodes']

In [11]:
%%time
fitlered_gene=pd.DataFrame()
for i in combined.obs['AIFI_L3'].unique():
    print(i)
    adata_subset=combined[combined.obs['AIFI_L3']==i]
    sc.pp.filter_genes(adata_subset, min_cells=round(np.shape(adata_subset.X)[0]*0.1))
    gene_list=pd.DataFrame(list(adata_subset.var.index))
    gene_list.columns=['gene']
    gene_list['AIFI_L3']=i
    fitlered_gene=pd.concat([fitlered_gene,gene_list])

cDC1


/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/opt/conda/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:250: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['n_cells'] = number
/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/opt/conda/lib/python3.10/site-

KLRF1- effector Vd1 gdT
ISG+ CD16 monocyte


/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/opt/conda/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:250: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['n_cells'] = number
/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


CD14+ cDC2


...

/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/opt/conda/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:250: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['n_cells'] = number
/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/opt/conda/lib/python3.10/site-

DN T cell
ASDC


/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/opt/conda/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:250: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['n_cells'] = number
/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/opt/conda/lib/python3.10/site-

Activated memory B cell
CLP cell
CPU times: user 1min 30s, sys: 58.5 s, total: 2min 28s
Wall time: 2min 29s


/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/opt/conda/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:250: ImplicitModificationWarning: Trying to modify attribute `.var` of view, initializing view as actual.
  adata.var['n_cells'] = number
/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/opt/conda/lib/python3.10/site-

In [12]:
fitlered_gene.to_csv('/home/jupyter/formal_BRIanalysis/DEG_Mehul/filtered_gene_list_for_immunevar_day0_7.csv')

In [8]:
combined

AnnData object with n_obs × n_vars = 5740156 × 33538
    obs: 'barcodes', 'batch_id', 'cell_name', 'cell_uuid', 'chip_id', 'hto_barcode', 'hto_category', 'n_genes', 'n_mito_umis', 'n_reads', 'n_umis', 'original_barcodes', 'pbmc_sample_id', 'pool_id', 'seurat_pbmc_type', 'seurat_pbmc_type_score', 'umap_1', 'umap_2', 'well_id', 'subject.biologicalSex', 'subject.ethnicity', 'subject.partnerCode', 'subject.race', 'subject.subjectGuid', 'cohort.cohortGuid', 'sample.visitName', 'sample.visitDetails', 'subject.birthYear', 'CMV.IgG.Serology.Result.Interpretation', 'BMI', 'predicted_doublet', 'doublet_score', 'AIFI_L1', 'AIFI_L2', 'AIFI_L3', 'AIFI_L3.5', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mito', 'log1p_total_counts_mito', 'pct_counts_mito'

In [9]:
combined=combined[combined.obs['AIFI_L1']=="B cell"].copy()

/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1113: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [10]:
combined

AnnData object with n_obs × n_vars = 511167 × 33538
    obs: 'barcodes', 'batch_id', 'cell_name', 'cell_uuid', 'chip_id', 'hto_barcode', 'hto_category', 'n_genes', 'n_mito_umis', 'n_reads', 'n_umis', 'original_barcodes', 'pbmc_sample_id', 'pool_id', 'seurat_pbmc_type', 'seurat_pbmc_type_score', 'umap_1', 'umap_2', 'well_id', 'subject.biologicalSex', 'subject.ethnicity', 'subject.partnerCode', 'subject.race', 'subject.subjectGuid', 'cohort.cohortGuid', 'sample.visitName', 'sample.visitDetails', 'subject.birthYear', 'CMV.IgG.Serology.Result.Interpretation', 'BMI', 'predicted_doublet', 'doublet_score', 'AIFI_L1', 'AIFI_L2', 'AIFI_L3', 'AIFI_L3.5', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_50_genes', 'pct_counts_in_top_100_genes', 'pct_counts_in_top_200_genes', 'pct_counts_in_top_500_genes', 'total_counts_mito', 'log1p_total_counts_mito', 'pct_counts_mito'

In [123]:
combined=sc.read_h5ad('flu_raw.h5ad')
combined.obs.index=combined.obs['barcodes']

In [124]:
combined=combined[combined.obs['AIFI_L3']==""]


/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1118: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):


In [125]:
sc.pp.normalize_total(combined, target_sum=1e4)
sc.pp.log1p(combined)
sc.pp.highly_variable_genes(combined, min_mean=0.0125, max_mean=3, min_disp=0.5)
combined.raw = combined
combined = combined[:, combined.var.highly_variable]
sc.pp.scale(combined, max_value=10)
sc.tl.pca(combined, svd_solver='arpack')
sc.pp.neighbors(combined, n_neighbors=50, n_pcs=30)
sc.tl.umap(combined)

/opt/conda/lib/python3.10/site-packages/scanpy/preprocessing/_normalization.py:169: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/opt/conda/lib/python3.10/site-packages/scanpy/preprocessing/_highly_variable_genes.py:215: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  disp_grouped = df.groupby('mean_bin')['dispersions']
/opt/conda/lib/python3.10/site-packages/anndata/_core/anndata.py:1118: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if not is_categorical_dtype(df_full[k]):
/opt/conda/lib/python3.10/site-packages/scanpy/preprocessing/_simple.py:842: UserWarning: Received a view of an AnnData. Making a copy.
  view_to_actual(adata)
/opt/conda/lib/python3.10/site-packag

  0%|          | 0/500 [00:00<?, ?it/s]

	completed  0  /  500 epochs
	completed  50  /  500 epochs
	completed  100  /  500 epochs
	completed  150  /  500 epochs
	completed  200  /  500 epochs
	completed  250  /  500 epochs
	completed  300  /  500 epochs
	completed  350  /  500 epochs
	completed  400  /  500 epochs
	completed  450  /  500 epochs


In [126]:
meta_data_noDO=meta_data[meta_data['sample.visitName'].isin(['Flu Year 1 Day 7','Flu Year 2 Day 0','Flu Year 2 Day 7'])]

In [127]:
filtered_all=pd.DataFrame()
for i in meta_data_noDO['pbmc_sample_id']:
    print(i)
    result = pd.read_parquet("/home/jupyter/BRI_Analysis/scRNA/Labels/"+i+'_L3_decision_matrix.parquet')
    set1 = set(combined.obs['barcodes'].tolist())
    set2 = set(result['barcodes'].tolist())
    # Find intersection
    intersection = set1 & set2
    filtered=result[result['barcodes'].isin(intersection)]
    filtered_all=pd.concat([filtered_all,filtered])

PB00007-01
PB00008-01
PB00009-01
PB00011-01
PB00013-01
PB00017-01
PB00018-01
PB00019-01
PB00020-01
PB00021-01
PB00024-01
PB00027-05
PB00028-05
PB00029-02
PB00032-05
PB00033-06
PB00143-01
PB00256-01
PB00335-05
PB00336-05
PB00340-01
PB00343-01
PB00344-01
PB00348-01
PB00354-01
PB00355-01
PB00358-01
PB00359-01
PB00360-01
PB00361-01
PB00367-01
PB00370-05
PB00371-03
PB00372-01
PB00373-01
PB00374-01
PB00375-01
PB00376-01
PB00378-01
PB00379-04
PB00380-01
PB00381-01
PB00382-01
PB00385-01
PB00389-01
PB00392-01
PB00395-01
PB00396-01
PB00399-01
PB00400-01
PB00502-01
PB00503-01
PB00505-01
PB00506-01
PB00508-01
PB00509-01
PB00511-01
PB00512-01
PB00513-02
PB00515-01
PB00517-01
PB00519-01
PB00521-01
PB00522-01
PB00524-01
PB00527-01
PB00528-01
PB00531-01
PB00534-01
PB00536-01
PB00538-01
PB00540-01
PB00542-01
PB00543-01
PB00544-01
PB00546-01
PB00547-01
PB00548-01
PB00550-01
PB00551-01
PB00553-01
PB00554-01
PB00555-01
PB00556-01
PB00559-01
PB00560-01
PB00567-01
PB00570-01
PB00571-01
PB00572-01
PB00574-01